<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# This notebook creates an image with additional Applications that can be used alongside Jupyter

## Content
1. [Configuration](#Configuration)
2. [Build Image](#Build-Image)


### Configuration

In [1]:
image_name = 'jupyter-with-apps'
folder_name = '/efs/shared/samples/notebooks/M-Admin/extra_user_apps/'

In [2]:
%cd $folder_name

/efs/shared/samples/notebooks/M-Admin/extra_user_apps


### Build Image

Lets see how  orbit build image works...

get our orbit env and team names

In [3]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
(env_name,team_name)

('test-env-public', 'lake-creator')

Repository name will be created from the image name prefixed by the env context.  Users are only able to manipulate ECR repos that start with 'orbit-{env_name}-users-'

In [4]:
repository_name = (f"orbit-{env_name}-users-{image_name}")
repository_name

'orbit-test-env-public-users-jupyter-with-apps'

In [5]:
!aws ecr delete-repository --repository-name $repository_name --force

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:495869084367:repository/orbit-test-env-public-users-jupyter-with-apps",
        "registryId": "495869084367",
        "repositoryName": "orbit-test-env-public-users-jupyter-with-apps",
        "repositoryUri": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-with-apps",
        "createdAt": 1617317152.0,
        "imageTagMutability": "MUTABLE"
    }
}


adding a file to our docker as an example

In [6]:
pwd = %pwd
pwd

'/efs/shared/samples/notebooks/M-Admin/extra_user_apps'

Now lets run the command

In [7]:
%%time

output = !orbit build image -e $env_name -d $pwd -n $image_name 
output

CPU times: user 12.5 ms, sys: 3.53 ms, total: 16 ms
Wall time: 7min 1s


['',
 'Deploying Docker Image |\x1b                   \x1b|   0% \x1b',
 '                                                  ',
 '',
 'Deploying Docker Image |\x1b▏                  \x1b|   1% \x1b',
 'Deploying Docker Image |\x1b▌                  \x1b|   3% \x1b',
 'Deploying Docker Image |\x1b▉                  \x1b|   5% \x1b',
 'Deploying Docker Image |\x1b███▊               \x1b|  20% \x1b',
 'Deploying Docker Image |\x1b█████▏             \x1b|  27% \x1b',
 'Deploying Docker Image |\x1b██████████████████▊\x1b|  99% \x1b',
 '                                                  ',
 '',
 'Deploying Docker Image |\x1b██████████████████▊\x1b|  99% \x1b',
 '                                                  ',
 '',
 'Deploying Docker Image |\x1b██████████████████▊\x1b|  99% \x1b',
 '                                                  ',
 '',
 'Deploying Docker Image |\x1b██████████████████▊\x1b|  99% \x1b',
 'Deploying Docker Image |\x1b███████████████████\x1b| 100% \x1b',
 '                

Lets get the image address from the output of the previous command

In [8]:
look_for = 'ECR Image Address='
image = None
for o in output:
    if look_for in o:
        image = o[o.index(look_for) + len(look_for):]
        print(image)

assert(image != None)       
    

495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-with-apps


In [9]:
# check that the image was built
import json
print(repository_name)
images = !aws ecr list-images --repository-name $repository_name
images = "".join(images)
im = json.loads(images)
print(im['imageIds'])
assert(len(im['imageIds']) > 0)

orbit-test-env-public-users-jupyter-with-apps
[{'imageDigest': 'sha256:56e19449229c0731d9294cffcdf7cf8b35b41107c25a1dc474b5df23ca4a3384', 'imageTag': 'latest'}]


### Building the profile for the Image

In [10]:
import json
profile = {
        "display_name": 'Workbench Apps (mid-size)',
        "description": "Build an image with additional data and development applications",
        "kubespawner_override": {
            "image": image,
            "cpu_guarantee": 4,
            "cpu_limit": 4,
            "mem_guarantee": "4G",
            "mem_limit": "4G"
        }
}

with open("profile.json", 'w') as f:
    json.dump(profile, f)


In [11]:
!cat profile.json

{"display_name": "Workbench Apps (mid-size)", "description": "Build an image with additional data and development applications", "kubespawner_override": {"image": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-with-apps", "cpu_guarantee": 4, "cpu_limit": 4, "mem_guarantee": "4G", "mem_limit": "4G"}}

In [12]:
!orbit build profile --debug --env $env_name --team lake-user profile.json

[2021-04-01 23:16:32,744][__main__.py  :481] env: test-env-public
[2021-04-01 23:16:32,744][__main__.py  :482] team: lake-user
[2021-04-01 23:16:32,744][__main__.py  :484] profile: {"display_name": "Workbench Apps (mid-size)", "description": "Build an image with additional data and development applications", "kubespawner_override": {"image": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-with-apps", "cpu_guarantee": 4, "cpu_limit": 4, "mem_guarantee": "4G", "mem_limit": "4G"}}
[2021-04-01 23:16:32,744][__main__.py  :485] debug: True
[2021-04-01 23:16:32,744][messages.py  :133] Progress bar: 1%
[ Info ] Retrieving existing profiles
[2021-04-01 23:16:32,744][image.py     : 35] Trying to read profiles from SSM parameter (/orbit/test-env-public/teams/lake-user/user/profiles).
[2021-04-01 23:16:32,885][image.py     :102] Existing user profiles for team lake-user: [{'description': 'Build an image with additional data and development applications', 'display_

## Users can now use the new profile to use new apps